# Select the top 3 products of each category

In [32]:
import pandas as pd

top3_csv = pd.read_csv("../data/reviews.csv")

top3_csv.head()

,asins,name,rating,title_text_processed,imageURLs,cluster_name
0,"B00QWO9P0O,B00LH3DMUO",AmazonBasics AAA Performance Alkaline Batterie...,3.0,... 3 one item bad quality missing backup spri...,https://images-na.ssl-images-amazon.com/images...,Smart Home & Amazon devices
1,"B00QWO9P0O,B00LH3DMUO",AmazonBasics AAA Performance Alkaline Batterie...,4.0,... always less expensive way go product like ...,https://images-na.ssl-images-amazon.com/images...,Smart Home & Amazon devices
2,"B00QWO9P0O,B00LH3DMUO",AmazonBasics AAA Performance Alkaline Batterie...,5.0,... duracell price happy well duracell price h...,https://images-na.ssl-images-amazon.com/images...,Smart Home & Amazon devices
3,"B00QWO9P0O,B00LH3DMUO",AmazonBasics AAA Performance Alkaline Batterie...,5.0,... well name brand battery much better seem w...,https://images-na.ssl-images-amazon.com/images...,Smart Home & Amazon devices
4,"B00QWO9P0O,B00LH3DMUO",AmazonBasics AAA Performance Alkaline Batterie...,5.0,... battery long lasting price great battery l...,https://images-na.ssl-images-amazon.com/images...,Smart Home & Amazon devices


### Group by ASIN and Aggregate Product-Level Info

In [ ]:
# Group by ASIN and aggregate product data
groups = top3_csv.groupby("asins").agg({
    "name": "first",  # Take first product name
    "cluster_name": lambda x: x.mode().iloc[0],  # Most frequent cluster name
    "rating": ["mean", "count"],
    "imageURLs": lambda x: ','.join(set(
        # Combine unique image URLs
        url for urls in x.dropna().astype(str) for url in urls.split(',')
    )),
})

#flatten multi-level column names and reset the index.
groups.columns = ["name", "cluster_name", "rating_mean", "rating_count", "imageURLs"]
groups = groups.reset_index()

# Round average rating to 2 decimals
groups["rating_mean"] = groups["rating_mean"].round(2)

groups.head()

,asins,name,cluster_name,rating_mean,rating_count,imageURLs
0,"B0002LCUZK,B010CEC6MI,B01B25NN64",Expanding Accordion File Folder Plastic Portab...,E-Reader & Office Tablets,5.00,9,https://i.ebayimg.com/thumbs/images/g/UOsAAOSw...
1,B001NIZB5M,Amazon Kindle Replacement Power Adapter (Fits ...,E-Reader & Office Tablets,2.80,5,https://images-na.ssl-images-amazon.com/images...
2,B005OOKNP4,AmazonBasics Bluetooth Keyboard for Android De...,E-Reader & Office Tablets,4.33,6,https://images-na.ssl-images-amazon.com/images...
3,B005PB2T0S,"Amazon Kindle Lighted Leather Cover,,,\r\nAmaz...",E-Reader & Office Tablets,4.00,5,
4,B005PB2T2Q,"Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16...",E-Reader & Office Tablets,3.67,6,


### Calculating Bayesian-Style Weighted Score

In [ ]:
# Calculate average

#average number of reviews per product
C = groups["rating_count"].mean()

#global average rating across all products
m = groups["rating_mean"].mean()

# Compute weighted score (Bayesian average)
groups["weighted_score"] = (
    (groups["rating_count"] / (groups["rating_count"] + C)) * groups["rating_mean"] 
    + (C / (groups["rating_count"] + C)) * m
)

# Get top 3 products per cluster
top3_per_cluster = (
    groups.sort_values(["cluster_name", "weighted_score"], ascending=[True, False])
    .groupby("cluster_name")
    .head(3)
    .reset_index(drop=True)
)

### Filtering Reviews for These Top Products

In [94]:
# Filter reviews for top products
reviews = top3_csv[["asins", "rating", "title_text_processed"]]
filtered_reviews = reviews[reviews["asins"].isin(top3_per_cluster["asins"])]

# Add title length column
filtered_reviews['title_length_chars'] = filtered_reviews['title_text_processed'].str.len()

# Count reviews per product
reviews_per_asin = filtered_reviews.groupby('asins').size().reset_index(name='count')
print(reviews_per_asin)

# Filter for medium length reviews (300-500 chars)
reviews_in_range = filtered_reviews[
    (filtered_reviews['title_length_chars'] >= 300) & 
    (filtered_reviews['title_length_chars'] <= 500)
]

filtered_reviews.head()

                    asins  count
0              B00IOY8XWQ    599
1              B00IOYAM4I     51
2   B00L9EPT8O,B01E6AO69U   5944
3              B00OQVZDJM   3175
4              B010CEHQTG    640
5   B018SZT3BK,B01AHB9CN2   2368
6              B018T075DC    455
7              B018Y226XO   1663
8              B01J94SBEY    233
9              B01J94SCAM    191
10             B01J94YIT6    831
11             B06XB29FPF    590


/var/folders/zf/m3g54d9d25q0jn4s7wbccll00000gn/T/ipykernel_32211/2559917676.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_reviews['title_length_chars'] = filtered_reviews['title_text_processed'].str.len()


,asins,rating,title_text_processed,title_length_chars
9673,B00IOY8XWQ,5.0,bit small useful bought save book bit small fi...,75
9674,B00IOY8XWQ,5.0,decent gift bought gift think pretty decent qu...,58
9675,B00IOY8XWQ,5.0,great e-reader great e-reader perhaps little p...,99
9676,B00IOY8XWQ,5.0,great kindle owned several kindles year one be...,144
9677,B00IOY8XWQ,5.0,great space saver book lover nothing replace j...,289


## Tagging Review Sentiment

In [96]:
# Categorize review sentiment
def categorize_rating(rating):
    if rating in [1, 2]:
        return "negative"
    elif rating == 3:
        return "neutral"
    return "positive"

# Apply sentiment to both DataFrames
filtered_reviews['sentiment'] = filtered_reviews['rating'].apply(categorize_rating)
reviews_in_range['sentiment'] = reviews_in_range['rating'].apply(categorize_rating)  # Fixed: use own ratings

# Count reviews by sentiment
rating_counts = filtered_reviews.groupby(['asins', 'sentiment']).size().reset_index(name='total_reviews')
range_counts = reviews_in_range.groupby(['asins', 'sentiment']).size().reset_index(name='reviews_title_300_500')

# Merge counts
final_counts = pd.merge(
    rating_counts,
    range_counts,
    on=['asins', 'sentiment'],
    how='left'
).fillna(0).astype({'reviews_title_300_500': int})

print(final_counts)

                    asins sentiment  total_reviews  reviews_title_300_500
0              B00IOY8XWQ  negative              7                      1
1              B00IOY8XWQ   neutral             12                      1
2              B00IOY8XWQ  positive            580                     38
3              B00IOYAM4I  positive             51                      4
4   B00L9EPT8O,B01E6AO69U  negative             79                      8
5   B00L9EPT8O,B01E6AO69U   neutral            209                     12
6   B00L9EPT8O,B01E6AO69U  positive           5656                    174
7              B00OQVZDJM  negative             30                      3
8              B00OQVZDJM   neutral             56                      4
9              B00OQVZDJM  positive           3089                    121
10             B010CEHQTG  negative             11                      2
11             B010CEHQTG   neutral             23                      3
12             B010CEHQTG  positive   

/var/folders/zf/m3g54d9d25q0jn4s7wbccll00000gn/T/ipykernel_32211/1726897627.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_reviews['sentiment'] = filtered_reviews['rating'].apply(categorize_rating)
/var/folders/zf/m3g54d9d25q0jn4s7wbccll00000gn/T/ipykernel_32211/1726897627.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_in_range['sentiment'] = reviews_in_range['rating'].apply(categorize_rating)  # Fixed: use own ratings


## Consolidating Sample Reviews by Sentiment

In [98]:
# Combine top 20 reviews per sentiment as strings
grouped_reviews = (
    reviews_in_range
    .groupby(['asins', 'sentiment'])
    ['title_text_processed']
    .apply(lambda x: ' '.join(x.head(20)))  # Join first 20 reviews
    .unstack()
    .rename(columns={
        'positive': 'positive_reviews',
        'negative': 'negative_reviews'
    })
    .reset_index()
)

# Merge with product data
final_df = pd.merge(
    top3_per_cluster,
    grouped_reviews[['asins', 'positive_reviews', 'negative_reviews']],
    on='asins',
    how='left'
)

In [99]:
final_df.to_csv("../data/top3_products.csv", index=False)